In [ ]:
!pip install talib-binary

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime as dt

import talib

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import os
df = pd.read_csv('../input/binanceeth/binanceeth.csv')

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, f1_score
from sklearn import model_selection


In [ ]:
def indicatorcokadosc(data, küçük, büyük):
    x=str(küçük)+"-"+str(büyük)
    adosc = talib.ADOSC(data.High, data.Low, data.Close, data.Volume, fastperiod=küçük, slowperiod=büyük)  # volume
    data = data.assign(adosc=adosc.values)
    isim = "adosc" + str(x)
    data.rename(columns={'adosc': isim}, inplace=True)
    adosc=''

    return data

In [ ]:
df.rename(columns = {'adj_close':'Adj Close','open':'Open','high':'High','low':'Low','close':'Close','volume':'Volume','Unnamed: 0':'Date'},inplace=True)
df.drop(['Adj Close','datetime'], axis=1,inplace=True)

In [ ]:
son=df
df=""

In [ ]:
son.Date=[str(x)[11:13] for x in son.Date]

In [ ]:
son=son.assign(ondkyüzde=son.Close.pct_change(periods=10))
son=son.assign(ondkyüzde=son.ondkyüzde.shift(periods=-10))

In [ ]:
def knnsonuc(son):
    işlemdata=son.drop(son.columns[:6], axis=1)
    işlemdata=işlemdata[15000:]
    işlemdata=işlemdata.dropna()
    işlemdata['ondkyüzde'] = işlemdata['ondkyüzde'].apply(lambda x: 1 if x <= 0 else 0)
    ydata=işlemdata.drop(işlemdata.columns[1:],axis=1)
    xdata=işlemdata.drop(işlemdata.columns[:1],axis=1)
    xdata.reset_index(drop=True,inplace=True)
    ydata.reset_index(drop=True,inplace=True)

    X_train, X_test, Y_train, Y_test = model_selection.train_test_split(xdata, ydata, test_size=0.033, random_state=123456789)#0.033,0.05
    xdata=""
    ydata=""
    X_test=X_test.reset_index(drop=True)
    Y_test=Y_test.reset_index(drop=True)
    
    eşitleme=len(Y_test[Y_test['ondkyüzde']== 1])-len(Y_test[Y_test['ondkyüzde']== 0])
    if eşitleme>=0:
        cıkılacak=Y_test[Y_test['ondkyüzde']== 1].head(eşitleme)
        cıkılacak=cıkılacak.index
        
        X_test=X_test.drop(cıkılacak)
        Y_test=Y_test.drop(cıkılacak)
        
    else:
        cıkılacak=Y_test[Y_test['ondkyüzde']== 0].head(eşitleme)
        cıkılacak=cıkılacak.index
        X_test=X_test.drop(cıkılacak)
        Y_test=Y_test.drop(cıkılacak)
        
    X_test=X_test.reset_index(drop=True)
    Y_test=Y_test.reset_index(drop=True)

    X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X_test, Y_test, test_size=0.3, random_state=123456789)

    knn=KNeighborsClassifier(n_neighbors=5)
    knn.fit(X_train,Y_train)
    
    y_pred = knn.predict(X_test)
    y_true = Y_test
    
    #print(confusion_matrix(y_true, y_pred))
    #print(classification_report(y_true, y_pred))
    
    ac = accuracy_score(y_true,y_pred)
    print(ac)
    return ac
    

In [ ]:
degeradosc=[]
plt.style.use("fivethirtyeight")

for büyük in range(10,1600,20):
    küçükdeg=[]
    for küçük in range(10,büyük-1,50):
        print("küçük="+str(küçük)+" büyük="+str(büyük))
        son=indicatorcokadosc(son, küçük, büyük)
        ort=knnsonuc(son)
        son=son.drop(son.columns[7:], axis=1)
        degeradosc=degeradosc+[ort]
        küçükdeg=küçükdeg+[ort]
    plt.plot(küçükdeg)
plt.show()

In [ ]:
degeradosc

In [ ]:
sns.lineplot(data=degeradosc)

In [ ]:
max(degeradosc)